In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [2]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator

In [3]:
df = pd.read_csv('../datasets/Amazon_employee.csv')

In [4]:
df.shape

(32769, 10)

In [5]:
bdf = BaseDatasetTransform(df,target='ACTION')

In [6]:
df,categorical_features = bdf.fit_transform()


В колонке ACTION нет пропущенных значений
-------------------------------------------
В наборе данных нет пропущенных значений
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index([], dtype='object')

Числовые колонки:
Index(['ACTION', 'RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2',
       'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY',
       'ROLE_CODE'],
      dtype='object')


In [7]:
X_train, X_test, y_train, y_test  = bdf.get_train_test_split()

Количество значений целевой переменной по категориям
ACTION
1    30872
0     1897
Name: count, dtype: int64
Следует ли выполнить стратифицированное раздеение на обучающую и тестовую выборку? y/n
Разделение датасета выполнено успешно


In [11]:
from sklearn.utils.class_weight import compute_class_weight 
#is_unbalance=True

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

class_counts = y_train.value_counts()


positive_class_count = class_counts[1]  
negative_class_count = class_counts[0]  

scale_pos_weight = negative_class_count / positive_class_count

lgbm_model = lgb.LGBMClassifier(verbose=-1,class_weight=class_weights_dict)


columns_to_delete = []  
columns_to_delete = []  


model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    objective='binary',
    metric='binary_error'
)

In [12]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics)

evaluator.fit(X_train, y_train)

ModelEvaluator(metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=[],
                                            columns_to_delete=[],
                                            model=LGBMClassifier(class_weight={0: 8.636295180722891,
                                                                               1: 0.5307265155020824},
                                                                 metric='binary_error',
                                                                 objective='binary',
                                                                 verbose=-1)))

In [13]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,Metric,Score
0,accuracy,0.852100
1,precision,0.602171
2,recall,0.759031
3,f1,0.627616
